<a href="https://colab.research.google.com/github/LeandroTinoco/Dynamic-Black-Litterman-Model/blob/main/The_10y_Dynamic_Black_Litterman_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy matplotlib yfinance PyPortfolioOpt
import os
if not os.path.isdir('data'):
    os.system('git clone https://github.com/robertmartin8/PyPortfolioOpt.git')
    os.chdir('PyPortfolioOpt/cookbook')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
pip install pyfiglet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00


In [ ]:
pip install tabulate

In [ ]:
import numpy as np
import pandas as pd
import math
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import yfinance as yf
from pypfopt import EfficientFrontier, objective_functions
from pypfopt.plotting import plot_weights
import plotly.express as px
import pypfopt
pypfopt.__version__
from pypfopt import black_litterman, risk_models, expected_returns
from pypfopt import BlackLittermanModel, plotting
from pypfopt.plotting import plot_weights
import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
from tabulate import tabulate
from pyfiglet import Figlet


In [ ]:
# ybill ={'2023': 4.72,
#   '2022': 0.40,
#   '2021': 0.10,
#   '2020': 1.56,
#   '2019': 2.60,
#   '2018': 1.83,
#   '2017': 0.81,
#   '2016': 0.61,
#   '2015': 0.25,
#   '2014': 0.13,
#   '2013': 0.15}

tickers = ["VGT","VHT","VFH","VCR","VIS","VOX","VDC","VDE","VAW", "VPU", "VNQ"]
estimate = [ 0.07 , 0.07 , 0.07 , 0.07 , 0.07 , 0.07 ,0.07 ,0.07 , 0.07 , 0.07 , 0.07]
confidences = [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0]
start_date = "2005-01-01"
market_prices_all = yf.download("VFIAX", start=start_date)["Close"]
sp_index_all = yf.download("SPY", start="2005-01-01")["Close"]

ohlc = yf.download(tickers, start=start_date)
#risk_free_rate = yf.download("^TNX", start=start_date)




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  11 of 11 completed


In [ ]:
#for the final print loop
years = []
longshort_values = []
maxsharp_values = []
benchmark_values = []


end_date = "2013-12-31"
format_string = "%Y-%m-%d"
start_date = "2005-01-01"
  # Open our new sheet and add some data.
#worksheet = gc.open(spredsheet_name).sheet1
result_1 = []
result_2 = []
bench = []
ret_1 = []
ret_2 = []
# Loop to add 1 year for 10 times
for _ in range(10):
  # Convert the end_date string to a datetime object
  end_date_dt = datetime.strptime(end_date, format_string)

  # Add 1 year to the datetime object
  end_date_dt += relativedelta(years = 1)  # Assuming a year has 365 days

  # Convert the datetime object back to a string
  end_date = end_date_dt.strftime(format_string)

  # Convert the end_date string to a datetime object
  start_date_dt = datetime.strptime(end_date, format_string)

  # Add 1 year to the datetime object
  start_date_dt -= relativedelta(years = 4)  # Assuming a year has 365 days

  # Convert the datetime object back to a string
  start_date_market = start_date_dt.strftime(format_string)

  prices = ohlc["Close"][(start_date):(end_date)]
  market_prices = market_prices_all[(start_date_market):(end_date)]
  mcaps_percent = [28.39 , 13.23 , 12.2 , 10.86 , 8.96 , 8.381 , 6.44 , 4.09 , 2.52 ,2.48 , 2.46]
  mcaps = {}
  counter = 0
  for t in tickers:
    stock = yf.Ticker(t)
    mcaps[t] = mcaps_percent[counter] * 1000000
    counter += 1

  #S = risk_models.CovarianceShrinkage(prices).ledoit_wolf(shrinkage_target='single_factor')
  #S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
  S = pypfopt.risk_models.risk_matrix(prices, method='exp_cov')
  #mu = expected_returns.mean_historical_return(prices)
  delta = black_litterman.market_implied_risk_aversion(market_prices)
  market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S,risk_free_rate=0.022)

  estimate = estimate
  viewdict = {}
  counter = 0
  for t in tickers:
    stock = yf.Ticker(t)
    viewdict[t] = estimate[counter]
    counter += 1

  bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

  confidences = confidences

  bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)

  lb = prices.mean() - prices.std()
  ub = prices.mean() + prices.std()

  intervals = []
  for i in range(len(lb)):
      intervals.append([lb[i], ub[i]])

      variances = []
  for lb, ub in intervals:
      sigma = (ub - lb)/2
      variances.append(sigma ** 2)

  omega = np.diag(variances)

  bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                          absolute_views=viewdict, omega=omega)

  ret_bl = bl.bl_returns()

  rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)],
              index=["Prior", "Posterior", "Views"]).T

  S_bl = bl.bl_cov()

  #Maximum Sharpe (No shorts allowed)
  ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(0,1))
  raw_weights_maxsharpe_exp = ef.max_sharpe(risk_free_rate=0.02)
  weights = ef.clean_weights()
  # pd.Series(weights).plot.pie(figsize=(4,4));
  #ef.portfolio_performance(verbose=True)

  #Max Sharpe with Shorts Allowed (Long Short)
  ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(-1,1))
  raw_weights_minvol_exp = ef.max_sharpe(risk_free_rate=0.02)
  weights = ef.clean_weights()
  # plot_weights(weights)
  #ef.portfolio_performance(verbose=True)

  # Convert the date string to a datetime object

  inicial_test_date = datetime.strptime(end_date, "%Y-%m-%d")

  # Add 2 days to the initial date
  inicial_test_date = inicial_test_date + timedelta(days=1)

  # Convert the new date back to a string
  inicial_test_date_string = inicial_test_date.strftime("%Y-%m-%d")

  # Convert the date string to a datetime object

  end_test_date = datetime.strptime(inicial_test_date_string, "%Y-%m-%d")

  # Add 1 year to the initial date
  end_test_date = end_test_date + relativedelta(years = 1)

  # Convert the new date back to a string
  end_test_date_string = end_test_date.strftime("%Y-%m-%d")

  #portfolio_returns= yf.download(tickers, start=(end_date))["Close"].pct_change().dropna()
  portfolio_returns= ohlc["Close"][(end_date):].pct_change().dropna()
  test = portfolio_returns[(inicial_test_date_string):(end_test_date)]

  #sp_index_all = yf.download("SPY", start=end_date, end=end_test_date)["Close"].pct_change().dropna()
  sp_index = sp_index_all[(end_date):(end_test_date)].pct_change().dropna()

  weights_minvol_exp = list(raw_weights_minvol_exp.values())
  weights_maxsharpe_exp = list(raw_weights_maxsharpe_exp.values())

  ret_1 = test.dot(weights_minvol_exp).add(1).cumprod()#.subtract(1).multiply(100)
  ret_2 = test.dot(weights_maxsharpe_exp).add(1).cumprod()#.subtract(1).multiply(100)
  ind_ret =  sp_index[(inicial_test_date_string):(end_test_date)].add(1).cumprod()#.subtract(1).multiply(100)
  #ind_ret.name = 'Benchmark (S&P500)'



  # print("last training date:", end_date)
  # print("last test date:", end_test_date_string)
  # print(inicial_test_date.year)
  # print("LongShort:", round(ret_1.iloc[-1], 2))
  # print("MaxSharp:", round(ret_2.iloc[-1], 2))
  # print("BENCHMARK:", round(ind_ret.iloc[-1], 2))
  # print()



  years.append(inicial_test_date.year)
  longshort_values.append(round(ret_1.iloc[-1], 2))
  maxsharp_values.append(round(ret_2.iloc[-1], 2))
  benchmark_values.append(round(ind_ret.iloc[-1], 2))


  result_1.append(ret_1.iloc[-1])
  result_2.append(ret_2.iloc[-1])
  bench.append(ind_ret.iloc[-1])
##LOOP END

custom_figlet = Figlet(font='big',width=70)  # You can choose a different font style
#custom_figlet_sub = Figlet(font='smslant', width =70)
logo_name = "S&P KILLER by L.Mileski "
#sub = ""
ascii_art = custom_figlet.renderText(logo_name)
#ascii_art_sub = custom_figlet_sub.renderText(sub)

print(ascii_art)
#print(ascii_art_sub)



data_matrix_compound = []

# Collect data for each row
data_matrix_compound.append(["LongShort",
                             f"{round(np.prod(result_1) * 100) - 100}%",
                             f"{round(np.prod(result_1[-7:]) * 100) - 100}%",
                             f"{round(np.prod(result_1[-5:]) * 100) - 100}%",
                             f"{round(np.prod(result_1[-3:]) * 100) - 100}%",
                             f"{round(np.prod(result_1[-1]) * 100) - 100}%"])

data_matrix_compound.append(["maxSharpe",
                             f"{round(np.prod(result_2) * 100) - 100}%",
                             f"{round(np.prod(result_2[-7:]) * 100) - 100}%",
                             f"{round(np.prod(result_2[-5:]) * 100) - 100}%",
                             f"{round(np.prod(result_2[-3:]) * 100) - 100}%",
                             f"{round(np.prod(result_2[-1]) * 100) - 100}%"])

data_matrix_compound.append(["Benchmark",
                             f"{round(np.prod(bench) * 100) - 100}%",
                             f"{round(np.prod(bench[-7:]) * 100) - 100}%",
                             f"{round(np.prod(bench[-5:]) * 100) - 100}%",
                             f"{round(np.prod(bench[-3:]) * 100) - 100}%",
                             f"{round(np.prod(bench[-1]) * 100) - 100}%"])


headers = ["CUMULATIVE", "10y", "7y", "5y", "3y", "1y"]
print(tabulate(data_matrix_compound, headers=headers, tablefmt="grid"))

# print("          AVERAGE")
# print("LongShort:",
#       round(np.mean(result_1), 2))

# print("maxSharpe:",
#       round(np.mean(result_2), 2))

# print("      Spy:",
#       round(np.mean(bench), 2))

data_matrix_average = []

# Convert values to percentages by multiplying by 100
longshort_mean_pct = round(np.mean(result_1) * 100, 2)-100
longshort_max_pct = round(np.max(result_1) * 100, 2)-100
longshort_min_pct = round(np.min(result_1) * 100, 2)-100

maxsharp_mean_pct = round(np.mean(result_2) * 100, 2)-100
maxsharp_max_pct = round(np.max(result_2) * 100, 2)-100
maxsharp_min_pct = round(np.min(result_2) * 100, 2)-100

bench_mean_pct = round(np.mean(bench) * 100, 2)-100
bench_max_pct = round(np.max(bench) * 100, 2)-100
bench_min_pct = round(np.min(bench) * 100, 2)-100

data_matrix_average.append(["longShort",
                            f"{longshort_mean_pct:.0f}%",
                            f"{longshort_max_pct:.0f}%",
                            f"{longshort_min_pct:.0f}%"])

data_matrix_average.append(["maxSharpe",
                            f"{maxsharp_mean_pct:.0f}%",
                            f"{maxsharp_max_pct:.0f}%",
                            f"{maxsharp_min_pct:.0f}%"])

data_matrix_average.append(["benchMark",
                            f"{bench_mean_pct:.0f}%",
                            f"{bench_max_pct:.0f}%",
                            f"{bench_min_pct:.0f}%"])


headers = ["10YEARS", "AVERAGE", "BEST", "WORST"]

print(tabulate(data_matrix_average, headers=headers, tablefmt="grid"))

# Separate loop for printing



data_matrix = []
for year, longshort, maxsharp, benchmark in zip(years, longshort_values, maxsharp_values, benchmark_values):
    # Access the numeric value from the Series using .item() if it's a single-element Series
    # or .iloc[-1] for the last element, or .values[0] for the first element.
    benchmark_value = benchmark.item() if isinstance(benchmark, pd.Series) and len(benchmark) == 1 else benchmark
    data_matrix.append([year, f"{longshort * 100 -100:.0f}%", f"{maxsharp * 100 -100:.0f}%", f"{benchmark_value * 100 - 100:.0f}%"])

headers = ["Year", "LongShort", "MaxSharp", "BenchMark"]

print(tabulate(data_matrix, headers=headers, tablefmt="grid"))





<ipython-input-10-32a58b3dcc42>:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intervals.append([lb[i], ub[i]])
<ipython-input-10-32a58b3dcc42>:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intervals.append([lb[i], ub[i]])
<ipython-input-10-32a58b3dcc42>:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intervals.append([lb[i], ub[i]])
<ipython-input-10-32a58b3dcc42>:74: FutureWarning: Series.__getitem__ treating keys as posi

  _____        _____    _  _______ _      _      ______ _____  
 / ____| ___  |  __ \  | |/ /_   _| |    | |    |  ____|  __ \ 
| (___  ( _ ) | |__) | | ' /  | | | |    | |    | |__  | |__) |
 \___ \ / _ \/\  ___/  |  <   | | | |    | |    |  __| |  _  / 
 ____) | (_>  < |      | . \ _| |_| |____| |____| |____| | \ \ 
|_____/ \___/\/_|      |_|\_\_____|______|______|______|_|  \_\
                                                               
                                                               
 _             _        __  __ _ _           _    _  
| |           | |      |  \/  (_) |         | |  (_) 
| |__  _   _  | |      | \  / |_| | ___  ___| | ___  
| '_ \| | | | | |      | |\/| | | |/ _ \/ __| |/ / | 
| |_) | |_| | | |____ _| |  | | | |  __/\__ \   <| | 
|_.__/ \__, | |______(_)_|  |_|_|_|\___||___/_|\_\_| 
        __/ |                                        
       |___/                                         

+--------------+-------+------+------+------+------+
| 

<ipython-input-10-32a58b3dcc42>:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intervals.append([lb[i], ub[i]])
<ipython-input-10-32a58b3dcc42>:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intervals.append([lb[i], ub[i]])


In [ ]:
import numpy as np
import pandas as pd

# Your returns data
data = [
    ["Year", "LongShort", "MaxSharp", "BenchMark"],
    [2015, "1%", "2%", "1%"],
    [2016, "14%", "14%", "12%"],
    [2017, "22%", "22%", "21%"],
    [2018, "-18%", "-7%", "-5%"],
    [2019, "37%", "36%", "31%"],
    [2020, "34%", "28%", "18%"],
    [2021, "32%", "31%", "29%"],
    [2022, "-29%", "-19%", "-18%"],
    [2023, "50%", "40%", "27%"],
    [2024, "49%", "25%", "26%"],
]

# Create a Pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# Convert percentage strings to numeric values
for col in ["LongShort", "MaxSharp", "BenchMark"]:
    df[col] = df[col].str.rstrip('%').astype('float') / 100

# Risk-free rate (you'll need to define this)
risk_free_rate = 0.02

# --- Calculations ---

def calculate_var(returns, confidence_level=0.95):
    """Calculates Value at Risk (VaR)."""
    return np.percentile(returns, 100 * (1 - confidence_level))

def calculate_sharpe_ratio(returns, risk_free_rate):
    """Calculates Sharpe Ratio."""
    excess_return = returns.mean() - risk_free_rate
    return excess_return / returns.std()

def calculate_max_drawdown(returns):
    """Calculates Maximum Drawdown."""
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min()

## Calculate risk measures for each portfolio
for portfolio in ["LongShort", "MaxSharp", "BenchMark"]:
    returns = df[portfolio]

    var = calculate_var(returns)
    sharpe_ratio = calculate_sharpe_ratio(returns, risk_free_rate)
    max_drawdown = calculate_max_drawdown(returns)
    std_dev = returns.std()  # Calculate standard deviation

    print(f"--- {portfolio} ---")
    print(f"VaR (95%): {var:.4f}")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    print(f"Maximum Drawdown: {max_drawdown:.4f}")
    print(f"Standard Deviation: {std_dev:.4f}")  # Print std_dev
    print("-" * 20)

--- LongShort ---
VaR (95%): -0.2405
Sharpe Ratio: 0.6347
Maximum Drawdown: -0.2900
Standard Deviation: 0.2710
--------------------
--- MaxSharp ---
VaR (95%): -0.1360
Sharpe Ratio: 0.7818
Maximum Drawdown: -0.1900
Standard Deviation: 0.1944
--------------------
--- BenchMark ---
VaR (95%): -0.1215
Sharpe Ratio: 0.7395
Maximum Drawdown: -0.1800
Standard Deviation: 0.1650
--------------------
